In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch as th
import torch.nn as nn

In [2]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")

In [59]:
class Network(nn.Module):
    def __init__(self, n_inputs: int, n_outputs: int, hidden_size: int = 128):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_outputs),
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x: th.Tensor) -> th.Tensor:
        return self.softmax(self.layers(x))

,Date,Trading_Price,1H_x1_0,1H_x2_0,1H_x3_0,1H_x4_0,1H_x5_0,1H_x1_1,1H_x2_1,1H_x3_1,...,1H_x1_14,1H_x2_14,1H_x3_14,1H_x4_14,1H_x5_14,1H_x1_15,1H_x2_15,1H_x3_15,1H_x4_15,1H_x5_15
0,2023-09-20 16:00:00+00:00,1.07353,0.001951,0.001138,0.000009,0.000149,0.002003,-0.000504,-0.000242,0.000056,...,-0.000299,0.000206,0.000253,0.000627,0.000028,0.000131,0.000159,0.000066,0.000122,0.000580
1,2023-09-20 17:00:00+00:00,1.07275,-0.000727,-0.000130,0.001008,0.000746,0.000270,0.001951,0.001138,0.000009,...,-0.000328,-0.000515,-0.000421,0.000440,0.000122,-0.000299,0.000206,0.000253,0.000627,0.000028
2,2023-09-20 18:00:00+00:00,1.06869,-0.003785,-0.000745,-0.004718,0.003799,0.001207,-0.000727,-0.000130,0.001008,...,-0.000084,-0.000290,0.000000,0.000234,0.000037,-0.000328,-0.000515,-0.000421,0.000440,0.000122
3,2023-09-20 19:00:00+00:00,1.06588,-0.002629,-0.003561,-0.001490,0.002861,0.000066,-0.003785,-0.000745,-0.004718,...,0.000215,0.000262,0.000009,0.000281,0.000243,-0.000084,-0.000290,0.000000,0.000234,0.000037
4,2023-09-20 20:00:00+00:00,1.06615,0.000253,-0.002498,-0.000741,0.000103,0.001060,-0.002629,-0.003561,-0.001490,...,0.000084,0.000852,-0.000506,0.001048,0.000833,0.000215,0.000262,0.000009,0.000281,0.000243


In [20]:
batch_size = 64

In [94]:
df = pd.read_csv("datasets/training/EURUSD.csv")
df["Lable"] = (df["Trading_Price"]-df["Trading_Price"].shift(1)).apply(lambda x: 1 if x > 0 else 0)
df = df.dropna()
features = th.from_numpy(df.drop(["Date", "Trading_Price", "Lable"], axis=1).to_numpy()).to(device=device).type(th.float32)
lable = th.from_numpy(df["Lable"].to_numpy()).to(device=device)
lable = nn.functional.one_hot(lable, num_classes=2).type(th.float32)
df.head()

,Date,Trading_Price,1H_x1_0,1H_x2_0,1H_x3_0,1H_x4_0,1H_x5_0,1H_x1_1,1H_x2_1,1H_x3_1,...,1H_x2_14,1H_x3_14,1H_x4_14,1H_x5_14,1H_x1_15,1H_x2_15,1H_x3_15,1H_x4_15,1H_x5_15,Lable
0,2023-09-20 16:00:00+00:00,1.07353,0.001951,0.001138,0.000009,0.000149,0.002003,-0.000504,-0.000242,0.000056,...,0.000206,0.000253,0.000627,0.000028,0.000131,0.000159,0.000066,0.000122,0.000580,0
1,2023-09-20 17:00:00+00:00,1.07275,-0.000727,-0.000130,0.001008,0.000746,0.000270,0.001951,0.001138,0.000009,...,-0.000515,-0.000421,0.000440,0.000122,-0.000299,0.000206,0.000253,0.000627,0.000028,0
2,2023-09-20 18:00:00+00:00,1.06869,-0.003785,-0.000745,-0.004718,0.003799,0.001207,-0.000727,-0.000130,0.001008,...,-0.000290,0.000000,0.000234,0.000037,-0.000328,-0.000515,-0.000421,0.000440,0.000122,0
3,2023-09-20 19:00:00+00:00,1.06588,-0.002629,-0.003561,-0.001490,0.002861,0.000066,-0.003785,-0.000745,-0.004718,...,0.000262,0.000009,0.000281,0.000243,-0.000084,-0.000290,0.000000,0.000234,0.000037,0
4,2023-09-20 20:00:00+00:00,1.06615,0.000253,-0.002498,-0.000741,0.000103,0.001060,-0.002629,-0.003561,-0.001490,...,0.000852,-0.000506,0.001048,0.000833,0.000215,0.000262,0.000009,0.000281,0.000243,1


In [80]:
lable

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [1.]], device='cuda:0')

In [44]:
features.shape, lable.shape

(torch.Size([6254, 80]), torch.Size([6254, 1]))

In [126]:
X_train, X_test, y_train, y_test = train_test_split(features, lable, test_size=0.2, random_state=42)

In [125]:
epochs = 100000
learning_rate = 0.001

model = Network(
    n_inputs=features.shape[1],
    n_outputs=2,
    hidden_size=128
).to(device=device)

lossfn = nn.CrossEntropyLoss()
optimiser = th.optim.Adam(model.parameters(), lr=learning_rate)

In [127]:
model.train()
for epoch in range(epochs):
    y_pred = model(X_train)
    loss = lossfn(y_pred, y_train)
    loss.backward()
    optimiser.step()
    optimiser.zero_grad()
    
    if epoch % 100 == 0:
        model.eval()
        with th.inference_mode():
            y_pred = model(X_test)
            
        acc = sum(y_test.argmax(dim=1).unsqueeze(dim=1).cpu() == th.argmax(y_pred, dim=1).unsqueeze(dim=1).cpu()) / len(y_pred)
        loss = lossfn(y_pred, y_test)
        
        print(f"Epoch: {epoch} | Loss: {loss} | Accuracy: {acc}")
        
    
    optimiser.zero_grad()

Epoch: 0 | Loss: 0.6929103136062622 | Accuracy: tensor([0.5180])
Epoch: 100 | Loss: 0.6920903921127319 | Accuracy: tensor([0.5140])
Epoch: 200 | Loss: 0.6511706113815308 | Accuracy: tensor([0.6867])
Epoch: 300 | Loss: 0.5591529607772827 | Accuracy: tensor([0.8601])
Epoch: 400 | Loss: 0.49166566133499146 | Accuracy: tensor([0.9440])
Epoch: 500 | Loss: 0.4512191414833069 | Accuracy: tensor([0.9592])
Epoch: 600 | Loss: 0.42545124888420105 | Accuracy: tensor([0.9664])
Epoch: 700 | Loss: 0.4077536165714264 | Accuracy: tensor([0.9752])
Epoch: 800 | Loss: 0.3952900767326355 | Accuracy: tensor([0.9680])
Epoch: 900 | Loss: 0.3858652710914612 | Accuracy: tensor([0.9768])
Epoch: 1000 | Loss: 0.3777618408203125 | Accuracy: tensor([0.9840])
Epoch: 1100 | Loss: 0.3717906177043915 | Accuracy: tensor([0.9848])
Epoch: 1200 | Loss: 0.3669593036174774 | Accuracy: tensor([0.9848])
Epoch: 1300 | Loss: 0.36295583844184875 | Accuracy: tensor([0.9848])
Epoch: 1400 | Loss: 0.3602936863899231 | Accuracy: tensor

KeyboardInterrupt: 